In [10]:
# setup + import
from utils import *
import os
import sklearn
import pandas as pd
import pickle

# setup + import
data = pd.read_csv('../data_processed/loan-10k_lrn.csv')
data.head()

pickle_file = open('../data_processed/loan_column_types.pkl', 'rb')
feature_structure = pickle.load(pickle_file)
pickle_file.close()

In [ ]:
feature_columns = feature_structure['bin'] + feature_structure['cat'] + feature_structure['cont'] + feature_structure['ord']

feature_structure["cat"]
TARGET = feature_structure["target"]


X = data[feature_columns]
y = data[TARGET]

values = [v  for _,v in feature_structure.items()]
# pd.DataFrame({"type":[feature_structure.keys()], "columns":  values} )
pd.DataFrame({"type":feature_structure.keys(), "columns":  values} )

,type,columns
0,bin,"[term, pymnt_plan, initial_list_status, applic..."
1,cat,"[home_ownership, verification_status, loan_sta..."
2,ord,"[emp_length, issue_d_month, issue_d_year, earl..."
3,cont,"[loan_amnt, funded_amnt, funded_amnt_inv, int_..."
4,target,grade


In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

classifiers0 = [
    SVC(kernel='linear', random_state=42, decision_function_shape='ovo', probability=True),
    KNeighborsClassifier(n_neighbors=3, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
]

classifier1 = [ KNeighborsClassifier(n_jobs=-1)]
model1 = get_pipeline(feature_structure,classifier1[0])

# single model training
res_holdout, model_holdout = perform_holdout(X, y, model1, 42)
res_cv, model_cv = perform_cv(X, y, model1)

results = append_results({}, model_holdout, model_cv, res_holdout, res_cv)
pd.DataFrame(results)

Evaluating multiple models

In [ ]:
evaluate_models(data, feature_structure, classifiers0)

Comparison of SVC over kernel functions:

In [ ]:
classifiersSVC = [
    SVC(kernel='linear', random_state=42),
    SVC(kernel='rbf', random_state=42),
    SVC(kernel='poly', random_state=42),
    SVC(kernel='sigmoid', random_state=42),
]

resultsSVC = evaluate_models(data, feature_structure, classifiersSVC)
resultsSVC

Comparison of KNeighbours over k 

In [ ]:
classifierKneighbors = [ KNeighborsClassifier(n_jobs=-1, n_neighbors=k) for k in range(2, 13)]
resultsKNeighbors = evaluate_models(data, feature_structure, classifierKneighbors)
resultsKNeighbors

Decision Tree comparison over max_deph

In [ ]:
classifierDecisionTree = [ DecisionTreeClassifier(random_state=42, max_depth=depth) for depth in range(3, 15)]
resultsDecisionTree = evaluate_models(data, feature_structure, classifierDecisionTree)
resultsDecisionTree

In [18]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

test_data = pd.read_csv('../data_processed/loan-10k_tes.csv')
T = test_data[feature_columns]

classifier = DecisionTreeClassifier(random_state=42, max_depth=9)
model = get_pipeline(feature_structure,classifier)

# training
res_cv, model_cv = perform_cv(X, y, model)
# prediction
pred = model_cv.predict(T)

with open('../data_processed/kaggle-loan-prediction.csv', 'w', newline='') as file:
    for id_value, pred_value in zip(test_data["ID"], pred):
        file.write(f"{id_value},{pred_value}\n")
